<a href="https://colab.research.google.com/github/Yesaswi33/A-CRM-application-WHOLESALE-RICE-MILL/blob/main/mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === STEP 0: Mount Drive & List Files ===
from google.colab import drive
drive.mount('/content/drive')

BASE_DRIVE_PATH = '/content/drive/MyDrive/Mobilenet'
!ls "{BASE_DRIVE_PATH}"

# Look for train/valid folders and tfrecords:
!find "{BASE_DRIVE_PATH}" -maxdepth 2 -type f | grep ".tfrecord" || true


In [ ]:
# === STEP 1: Clone TF Models & Install Dependencies ===
!git clone --depth 1 https://github.com/tensorflow/models.git /content/models
!pip install -q tf_slim
!apt-get install -qq protobuf-compiler python3-dev
!cd /content/models/research && protoc object_detection/protos/*.proto --python_out=.
!cp /content/models/research/object_detection/packages/tf2/setup.py /content/models/research/
!cd /content/models/research && python3 -m pip install .
!python3 /content/models/research/object_detection/builders/model_builder_tf2_test.py


In [ ]:
# === STEP 2: Create label_map.pbtxt ===
class_names = [
  "allam_pachadi", "ariselu", "badam", "bagara_rice", "banana",
  "beerakaya_ridge_gourd_curry", "beetroot_curry", "bendakaya_fry",
  "biryani", "bonda", "carrot_fry", "cashew", "chapati",
  "cheapla_pulusu", "chegodilu", "chintapandu_pulihora", "chole",
  "coconut_chutney", "curd_rice_perugu_annam", "dondaka_curry",
  "dosa_minapattu", "egg", "fish_fry", "fresh_guavas", "garelu",
  "grapes", "idli", "jaggery_based_sweets", "jeera_rice",
  "kodi_pulusu", "laddoo", "mangoes", "milk", "mirapakaya_bajji",
  "mulakkaya_curry_drumsticks", "mutton_curry", "natu_ravva_upma",
  "nimmakaya_pulihora", "oranges", "palak_paneer", "palakura_pappu",
  "paneer", "paneer_curry", "papayas", "perugu", "pesarattu",
  "pistha", "poha", "pomegranates", "poori", "potato_curry",
  "potato_fry", "prawns_curry", "ragi_java", "rajma_curry",
  "rice", "sambar", "sambar_idli", "samosa", "thota_kura",
  "thota_kura_pappu", "tomato_pachadi", "tomato_pappu", "upma",
  "uthappam"
]

label_map_path = f"{BASE_DRIVE_PATH}/label_map.pbtxt"
with open(label_map_path, "w") as f:
    for idx, name in enumerate(class_names, start=1):
        f.write(f"item {{\n  id: {idx}\n  name: '{name}'\n}}\n\n")
print("Label map saved to:", label_map_path)


In [ ]:
# STEP 2: Create the destination directory in your Drive
!mkdir -p /content/drive/MyDrive/Mobilenet

# STEP 3: Download the pretrained model into your Drive
!wget -c "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz" \
    -O /content/drive/MyDrive/Mobilenet/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# STEP 4: Extract directly into the folder
!tar -xzf /content/drive/MyDrive/Mobilenet/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz \
    -C /content/drive/MyDrive/Mobilenet/

# STEP 5: Confirm it worked
import os
path = "/content/drive/MyDrive/Mobilenet/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
print("✅ Model folder exists:", os.path.exists(path))


In [ ]:
import re

# Paths to your files in Drive
base_path = "/content/drive/MyDrive/Mobilenet"
model_dir = f"{base_path}/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
config_path = f"{model_dir}/pipeline.config"
train_record = f"{base_path}/train/objects-Indian-Dishes-fish-fry.tfrecord"
val_record = f"{base_path}/valid/objects-Indian-Dishes-fish-fry.tfrecord"
label_map = f"{base_path}/label_map.pbtxt"
checkpoint_path = f"{model_dir}/checkpoint/ckpt-0"

# Number of classes
class_names = [  # shortened list for readability
    'allam_pachadi', 'ariselu', 'badam', 'bagara_rice', 'banana',
    'beerakaya_ridge_gourd_curry', 'beetroot_curry', 'bendakaya_fry', 'biryani',
    'bonda', 'carrot_fry', 'cashew', 'chapati', 'cheapla_pulusu', 'chegodilu',
    'chintapandu_pulihora', 'chole', 'coconut_chutney', 'curd_rice_perugu_annam',
    'dondaka_curry', 'dosa_minapattu', 'egg', 'fish_fry', 'fresh_guavas',
    'garelu', 'grapes', 'idli', 'jaggery_based_sweets', 'jeera_rice', 'kodi_pulusu',
    'laddoo', 'mangoes', 'milk', 'mirapakaya_bajji', 'mulakkaya_curry_drumsticks',
    'mutton_curry', 'natu_ravva_upma', 'nimmakaya_pulihora', 'oranges',
    'palak_paneer', 'palakura_pappu', 'paneer', 'paneer_curry', 'papayas',
    'perugu', 'pesarattu', 'pistha', 'poha', 'pomegranates', 'poori',
    'potato_curry', 'potato_fry', 'prawns_curry', 'ragi_java', 'rajma_curry',
    'rice', 'sambar', 'sambar_idli', 'samosa', 'thota_kura', 'thota_kura_pappu',
    'tomato_pachadi', 'tomato_pappu', 'upma', 'uthappam'
]
num_classes = len(class_names)

# Modify pipeline.config
with open(config_path, 'r') as file:
    config = file.read()

config = re.sub(r'num_classes: \d+', f'num_classes: {num_classes}', config)
config = re.sub(r'fine_tune_checkpoint: ".*?"', f'fine_tune_checkpoint: "{checkpoint_path}"', config)
config = re.sub(r'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "detection"', config)
config = re.sub(r'input_path: ".*?"', f'input_path: "{train_record}"', config, count=1)
config = re.sub(r'input_path: ".*?"', f'input_path: "{val_record}"', config)
config = re.sub(r'label_map_path: ".*?"', f'label_map_path: "{label_map}"', config)

# Save updated config
with open(config_path, 'w') as file:
    file.write(config)

print("✅ pipeline.config updated successfully.")


In [ ]:
!pip install -q tf-slim pycocotools
!pip install -q tensorflow-object-detection-api


In [ ]:
%cd /content
!git clone https://github.com/tensorflow/models.git


In [ ]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .


In [ ]:
!python object_detection/builders/model_builder_tf2_test.py


In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
  --model_dir="/content/drive/MyDrive/Mobilenet/output" \
  --pipeline_config_path="/content/drive/MyDrive/Mobilenet/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config" \
  --num_train_steps=10000 \
  --sample_1_of_n_eval_examples=1 \
  --alsologtostderr


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/Mobilenet/output


In [ ]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.


In [ ]:
!cp object_detection/packages/tf2/setup.py .
!pip install .


In [ ]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder

# Define paths
pipeline_config_path = "/content/drive/MyDrive/Mobilenet/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config"
checkpoint_path = "/content/drive/MyDrive/Mobilenet/output/checkpoint"
label_map_path = "/content/drive/MyDrive/Mobilenet/label_map.pbtxt"

# Load pipeline config and build the detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt_path = os.path.join(checkpoint_path, 'ckpt-0')  # Make sure 'ckpt-0.index' exists
ckpt.restore(ckpt_path).expect_partial()

print("✅ Model loaded successfully from checkpoint!")


In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load label map
category_index = label_map_util.create_category_index_from_labelmap(label_map_path)

# Load and preprocess image
image_path = "/content/test_image.jpg"  # replace with your image path
image_np = cv2.imread(image_path)
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

# Run detection
detections = detection_model(input_tensor)

# Process output
num_detections = int(detections.pop('num_detections'))
detections = {k: v[0, :num_detections].numpy() for k, v in detections.items()}
detections['num_detections'] = num_detections
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

# Visualize
image_np_with_detections = image_np.copy()
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes'],
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    min_score_thresh=0.4,
    line_thickness=3
)

# Show image
plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
